In [1]:
import tkinter as tk
from tkinter import messagebox
from PIL import Image, ImageTk
from tkinter import ttk
from datetime import datetime
from stable_baselines3 import A2C, TD3, PPO
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import itertools 

from finrl.meta.preprocessor.yahoodownloader import YahooDownloader 
from finrl import config_tickers 
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from stable_baselines3.common.logger import configure

from finrl import config
from finrl import config_tickers
from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.config_tickers import DOW_30_TICKER
from finrl.meta.data_processor import DataProcessor
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.meta.preprocessor.preprocessors import data_split
from finrl.meta.preprocessor.preprocessors import FeatureEngineer
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
import sys
import itertools

def transform_value(x):
    if x > 0:
        return 'buy'
    elif x < 0:
        return 'sell'
    else:
        return 'hold'
    
def is_number(s):
    try:
        float(s)  # Try converting the string to float
        return True
    except ValueError:
        return False

def process_entry(input_string):
    # Split the string into a list at each comma
    input_list = input_string.split(',')
    # Optionally, strip whitespace and convert each element as necessary
    input_list = [int(elem.strip()) for elem in input_list]
    return input_list

def show_next_row(tree, tree2, tree3, tree4, rows, rows2, rows3, rows4, current_row_index):
    
    # Insert the next row if available
    if current_row_index[0] < len(rows):
        tree.insert("", tk.END, values=rows[current_row_index[0]])
        current_row_index[0] += 1
        
    for i in tree.get_children():
        tree.delete(i)
        
    for i in tree2.get_children():
        tree2.delete(i)
        
    for i in tree3.get_children():
        tree3.delete(i)
    
    for i in tree4.get_children():
        tree4.delete(i)
    
    if current_row_index[0] < len(rows):
        values = rows[current_row_index[0]]
        values2 = rows2[current_row_index[0]]
        values3 = rows3[current_row_index[0]]
        values4 = rows4[current_row_index[0]]
        # Combine values and tags for insertion
        tree.insert("", tk.END, values=values)
        tree2.insert("", tk.END, values=values2)
        tree3.insert("", tk.END, values=values3)
        tree4.insert("", tk.END, values=values4)
        current_row_index[0] += 1

def add():
    try:
        print('he llegado 1')
        transaction_cost = float(entry1.get()) / 100
        print('he llegado 2')
        amount = float(entry2.get())
        maxk = float(entry3.get())
        print('he llegado 3')
        descubrir = entry4.get()
        es_numero = is_number(descubrir)
        
        if es_numero:
            ownedstock = float(descubrir)/29
            if ownedstock < 0:
                messagebox.showerror("Format error", "Please, introduce a valid number of shares (>=0)")
                return
        else:    
            lista_ownedstock = process_entry(descubrir)
    
        print('he llegado 4')
        startdate = str(entry5.get()) 
        enddate = str(entry6.get())
        print('he llegado 5')
        if datetime.strptime(startdate, "%Y-%m-%d") > datetime.strptime(enddate, "%Y-%m-%d"):
            messagebox.showerror("Format error", "The start date needs to be smaller than the end date")
            return
        print('he llegado 6')
        TRAIN_START_DATE = '2010-01-01'
        TRAIN_END_DATE = '2021-10-01'
        TEST_START_DATE = startdate
        TEST_END_DATE = enddate
        print('he llegado 7')
        stock = YahooDownloader(start_date = TRAIN_START_DATE,
                     end_date = TEST_END_DATE,
                     ticker_list = config_tickers.DOW_30_TICKER).fetch_data()

        tickers = stock.tic.unique()

        stock_data = stock.drop(columns=['day']).copy(deep=True) # independent copy with dates, prices, volumes and stock symbol (company)
        stock_data['date'] = pd.to_datetime(stock_data['date']) # convert date to type date
        stock_data_training = stock_data[stock_data['date'] < TEST_START_DATE]
        stock_data_testing = stock_data[stock_data['date'] >= TEST_START_DATE]
        stock_data.set_index('date', inplace=True) # set column date as index
        stock_data_training.set_index('date', inplace=True)
        stock_data_testing.set_index('date', inplace=True)
        
        INDICATORS = ['supertrend','high_5_sma','tema','chop']

        # creamos instancia de FeatureEngineer indicando la configuración deseada para hacer el preproceso
        fe = FeatureEngineer(
                            use_technical_indicator=True,
                            tech_indicator_list = INDICATORS,
                            use_vix=True, 
                            use_turbulence=True, # measures extreme asset price fluctuation --> if the turbulence index reaches a pre-defined threshold, the agent will halt buying action and start selling the holding shares gradually
                            user_defined_feature = False) 

        processed = fe.preprocess_data(stock)

        list_ticker = processed["tic"].unique().tolist()

        list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))

        combination = list(itertools.product(list_date,list_ticker))

        processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")

        processed_full = processed_full[processed_full['date'].isin(processed['date'])]

        processed_full = processed_full.sort_values(['date','tic']) # we need to sort bc of the data_split method CHECK SOURCE CODE 

        processed_full = processed_full.fillna(0)
        
        large_value = 1e9

        processed_full.replace([np.inf], large_value, inplace=True)
        train = data_split(processed_full, TRAIN_START_DATE,TRAIN_END_DATE)
        trade = data_split(processed_full, TEST_START_DATE,TEST_END_DATE)
        train_length = len(train)
        trade_length = len(trade)
        stock_dimension = len(train.tic.unique())
        state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
        
        buy_cost_list = sell_cost_list = [transaction_cost] * stock_dimension
        if es_numero:
            num_stock_shares = [ownedstock] * stock_dimension
        else:
            num_stock_shares = lista_ownedstock
        h_max = maxk
        initial_amount = amount
        
        gamma = 1e-4
        
        env_kwargs = {
            "hmax": h_max,
            "initial_amount": initial_amount, 
            "num_stock_shares": num_stock_shares,
            "buy_cost_pct": buy_cost_list,
            "sell_cost_pct": sell_cost_list,
            "state_space": state_space,
            "stock_dim": stock_dimension,
            "tech_indicator_list": INDICATORS,
            "action_space": stock_dimension,
            "reward_scaling": gamma
        }

        e_trade_gym = StockTradingEnv(df = trade, turbulence_threshold = 70,risk_indicator_col='vix', **env_kwargs)
        env_trade, obs_trade = e_trade_gym.get_sb_env()

        trained_model_a2c = A2C.load("/Users/carolinaalbamaruganrubio/ai4finance/FinRL/examples/resultadosTFG/definitivo/a2c_model.zip") #trained_a2c
        df_account_value_a2c, df_actions_a2c = DRLAgent.DRL_prediction(
            model=trained_model_a2c, 
            environment = e_trade_gym)
        
        df1 = df_actions_a2c
        
        trained_model_td3 = TD3.load("/Users/carolinaalbamaruganrubio/ai4finance/FinRL/examples/resultadosTFG/definitivo/td3_model.zip")
        df_account_value_td3, df_actions_td3 = DRLAgent.DRL_prediction(
            model=trained_model_td3, 
            environment = e_trade_gym)
        
        df2 = df_actions_td3
        
        trained_model_ppo = PPO.load("/Users/carolinaalbamaruganrubio/ai4finance/FinRL/examples/resultadosTFG/definitivo/ppo_model.zip")
        df_account_value_ppo, df_actions_ppo = DRLAgent.DRL_prediction(
            model=trained_model_ppo, 
            environment = e_trade_gym)
        
        df3 = df_actions_ppo
        
        df_temp = df1.add(df2).div(2) # AVERAGE RECOMMENDATION
        df = df_temp.add(df3).div(2).reset_index() # AVERAGE RECOMMENDATION
        
        df_actions = df.copy()
        for column in df.columns[1:]: 
            df_actions[column] = df[column].apply(transform_value)
            
        cols = df1.columns#[1:]  # Esto asume que 'ID' es siempre la primera columna
        # Calcular el máximo y mínimo sin afectar la columna 'ID'
        max_values = np.maximum.reduce([df1[cols].values, df2[cols].values, df3[cols].values])
        min_values = np.minimum.reduce([df1[cols].values, df2[cols].values, df3[cols].values])

        # Crear DataFrames de resultados con 'ID'
        max_df = pd.DataFrame(max_values, columns=cols)
        max_df.index = df1.index
        max_df = max_df.reset_index()
        #max_df.insert(0, 'date', df1['date'])

        min_df = pd.DataFrame(min_values, columns=cols)
        min_df.index = df1.index
        min_df = min_df.reset_index()
        #min_df.insert(0, 'date', df1['date'])
        
        root = tk.Tk()
        root.title("ROBOT ADVISOR")

        frame = tk.Frame(root)
        frame.pack(fill='both', expand=True)
        
        # Add a label for text
        label = tk.Label(frame, text="These are the orders I advise you to do for today:                                                                                                                                                                                                             ------->", font=('Helvetica', 16))
        label.pack(side='top')

        # Create the Treeview widget
        tree_2 = ttk.Treeview(frame, columns=list(df_actions.columns), show='headings', height=2)
        tree_2.pack(fill='both', expand=True)
        
        label2 = tk.Label(frame, text="For every order, this is the recommended amount of shares:                                                                                                                                                                                               ------->", font=('Helvetica', 16))
        label2.pack(side='top')
        
        tree = ttk.Treeview(frame, columns=list(df.columns), show='headings', height=2)
        tree.pack(fill='both', expand=True)
        
        label3 = tk.Label(frame, text="Or you can trade the following shares at the most:                                                                                                                                                                                                                       ------->", font=('Helvetica', 16))
        label3.pack(side='top')
        
        tree3 = ttk.Treeview(frame, columns=list(max_df.columns), show='headings', height=2)
        tree3.pack(fill='both', expand=True)
        
        label4 = tk.Label(frame, text="and the following shares at the least:                                                                                                                                                                                                                                 ------->", font=('Helvetica', 16))
        label4.pack(side='top')
        
        tree4 = ttk.Treeview(frame, columns=list(min_df.columns), show='headings', height=2)
        tree4.pack(fill='both', expand=True)
        
        label5 = tk.Label(frame, text="My recommendation is based on the best combination of technical indicators I was tested on, and the three best reinforcement learning algorithms resulting from the experiment.", font=('Helvetica', 12, 'italic'))
        label5.pack(side='top')

        # Define headings
        for col in df.columns:
            tree.heading(col, text=col)
            tree.column(col, anchor='center')
            
        for col in df_actions.columns:
            tree_2.heading(col, text=col)
            tree_2.column(col, anchor='center') 
            
        for col in df.columns:
            tree3.heading(col, text=col)
            tree3.column(col, anchor='center')
            
        for col in df.columns:
            tree4.heading(col, text=col)
            tree4.column(col, anchor='center')

        # Prepare a list of rows to be displayed
        rows = df.values.tolist()
        rows_2 = df_actions.values.tolist()
        rows3 = max_df.values.tolist()
        rows4 = min_df.values.tolist()
        current_row_index = [0]  # Use a list to maintain a reference in the nested function
        
        # Initially display the first row
        show_next_row(tree, tree_2, tree3, tree4, rows, rows_2, rows3, rows4, current_row_index)

        # Add a button to load the next row
        next_row_button = tk.Button(root, text="Next day!", command=lambda: show_next_row(tree, tree_2,  tree3, tree4, rows, rows_2, rows3, rows4, current_row_index))
        next_row_button.pack(side='bottom')
        
        scrollbar = ttk.Scrollbar(frame, orient="vertical")
        scrollbar.pack(side='right', fill='y')
        scrollbar.config(command=lambda *args: [tree.yview(*args), tree_2.yview(*args)])

        # Configure treeviews to update the scrollbar
        tree.config(yscrollcommand=scrollbar.set)
        tree_2.config(yscrollcommand=scrollbar.set)

        root.mainloop()
        
    except ValueError:
        messagebox.showerror("Input error", "Please enter valid numbers")

# Create the main window
root = tk.Tk()
root.title("ROBOT ADVISOR")

background_image = Image.open("/Users/carolinaalbamaruganrubio/ai4finance/FinRL/examples/newdesign.png")
background_photo = ImageTk.PhotoImage(background_image)

# Create a canvas to place the background image
canvas = tk.Canvas(root, width=background_photo.width(), height=background_photo.height())
canvas.pack(fill="both", expand=True)
canvas.create_image(0, 0, image=background_photo, anchor="nw")

# TEXT LABEL -----> Transaction Cost
label1 = tk.Label(root, text="How much does your trading agency charge you for every transaction? (in (%) and use . as decimal):", bg='white')
canvas.create_window(350, 51, window=label1)
# TEXT ENTRY for USER
entry1 = tk.Entry(root, bg='white', fg='black')
canvas.create_window(800, 50, window=entry1)

# TEXT LABEL -----> Amount $ 
label2 = tk.Label(root, text="What is the maximum amount of money you are willing to invest? (in $):", bg='white')
canvas.create_window(260, 76, window=label2)
# TEXT ENTRY for USER
entry2 = tk.Entry(root, bg='white', fg='black')
canvas.create_window(800, 75, window=entry2)

# TEXT LABEL -----> Max k trades
label3 = tk.Label(root, text="What is the maximum amount of trades you want to make in a day? (in number):", bg='white')
canvas.create_window(285, 101, window=label3)
# TEXT ENTRY for USER
entry3 = tk.Entry(root, bg='white', fg='black')
canvas.create_window(800, 100, window=entry3)

# TEXT LABEL -----> Already owned shares
label4 = tk.Label(root, text="How many shares from DOW-30 do you already own? ", bg='white')
canvas.create_window(207, 126, window=label4)
labelaclaracion = tk.Label(root, text="Please enter your total DOW-30 shares for an approximate prediction (in number)", bg='white', font=('Helvetica', 10, 'italic', 'bold'))
canvas.create_window(260, 151, window=labelaclaracion)
labelaclaracion = tk.Label(root, text="Optionally, for a more precise prediction, enter the number of shares you own for each company, separated by commas, in this order:", bg='white', font=('Helvetica', 9, 'italic', 'bold'))
canvas.create_window(375, 171, window=labelaclaracion)
labelordenstock = tk.Label(root, text="AXP,AMGN,AAPL,BA,CAT,CSCO,CVX,GS,HD,HON,IBM,INTC,JN,KO,JPM,MCD,MMM,MRK,MSFT,NKE,PG,TRV,UNH,CRM,VZ,V,WBA,WMT,DIS", bg='white', font=('Helvetica', 9, 'italic', 'bold'))
canvas.create_window(380, 191, window=labelordenstock)
# TEXT ENTRY for USER
entry4 = tk.Entry(root, bg='white', fg='black')
canvas.create_window(800, 161, window=entry4)

# TEXT LABEL -----> Date START
label5 = tk.Label(root, text="Which day do you want me to start advising you? (in 'yyyy-mm-dd'):", bg='white')
canvas.create_window(250, 226, window=label5)
# TEXT ENTRY for USER
entry5 = tk.Entry(root, bg='white', fg='black')
canvas.create_window(800, 226, window=entry5)

# TEXT LABEL -----> Date END
label6 = tk.Label(root, text="Which day do you want me to stop advising you? (in 'yyyy-mm-dd') (at least 7 days of difference):", bg='white')
canvas.create_window(342, 249, window=label6)
# TEXT ENTRY for USER
entry6 = tk.Entry(root, bg='white', fg='black')
canvas.create_window(800, 248, window=entry6)

# Create and place the buttons on the canvas
btn_add = tk.Button(root, text="Predict!", command=add, bg='green', fg='black', width=10, height=1)
canvas.create_window(800, 283, window=btn_add)

# Run the application
root.mainloop()

he llegado 1
he llegado 1
he llegado 2
he llegado 3
he llegado 4
he llegado 5
